## 라이브러리 선언

In [7]:
### 데이터 처리 라이브러리
import pandas as pd
import numpy as np

In [8]:
### 머신러닝 라이브러리
from sklearn.svm import SVC ## 서포트 vector
from sklearn.neighbors import KNeighborsClassifier # k neighbors

In [9]:
### 데이터 전처리 라이브러리
from sklearn.model_selection import train_test_split

In [10]:
### 클래시피케이션 matricx// 분류에 쓰이는 정확도 지표들 ->
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

#### Classification -  SVM(soft vector machine) 마진에 따라 마진에 있는 데이터만 남기고 계산 ( 계산량이 줄어듬, 단 차원 증가시 계산량 증가)

##### 정확도와 속도로 알고리즘끼리 경합을 시킨다

#### Classification - KNN 데이터로부터 가장 인접한 이웃개수의 그룹을 확인 후 편성

##### 느리다

## 데이터 불러오기

In [15]:
csData = pd.read_csv("../dataset/customer.csv")

In [16]:
csData.head(2)

,balance,stock,label
0,30000000,22500000,normal
1,280000000,48000000,diamond


#### balance -> 예금 보유금액
#### stock ->  주식 보유금색
#### label -> 고객등급. normal, diamon, vip

In [14]:
csData.label.drop_duplicates()

0     normal
1    diamond
4        vip
Name: label, dtype: object

## 1. 타입 통합 / 숫자형 컬럼 추가

In [19]:
csData.dtypes

balance     int32
stock       int32
label      object
dtype: object

In [18]:
csData = csData.astype({"balance" : "int", "stock" : "int", "label" : "object"})

In [20]:
csData.dtypes

balance     int32
stock       int32
label      object
dtype: object

## 2. 특성 선정

In [31]:
### 정답지
labelColumn = "label_new"

In [32]:
## 상관계수 생성 함수는 숫자형 컬럼에 대해서만 상관계수를 계산한다. 
csData.corr() ## 관계를 알 수 없다. 라벨이 나오지 않았다..


,balance,stock,LABEL_NEW
balance,1.000000,0.565942,0.883144
stock,0.565942,1.000000,0.824174
LABEL_NEW,0.883144,0.824174,1.000000


In [33]:
## 라벨인코더는 원하는 값으로 맵핑하기가 힘들다. 딕셔너리는 원하는 값으로 맵핑 가능

In [40]:
labelMap = {"normal" : 0, "diamond" : 1, "vip" : 2 }

In [41]:
csData[labelColumn] = csData.label.map(labelMap)

In [36]:
csData.corr()

,balance,stock,LABEL_NEW,label_new
balance,1.000000,0.565942,0.883144,0.883144
stock,0.565942,1.000000,0.824174,0.824174
LABEL_NEW,0.883144,0.824174,1.000000,1.000000
label_new,0.883144,0.824174,1.000000,1.000000


In [39]:
csData.drop(columns=["label_new"],inplace=True)
csData

,balance,stock,label
0,30000000,22500000,normal
1,280000000,48000000,diamond
2,300000000,40666666,diamond
3,54000000,28000000,normal
4,768000000,32000000,vip
...,...,...,...
19995,628000000,44666666,diamond
19996,276000000,20000000,normal
19997,652000000,41333333,diamond
19998,676000000,45333333,diamond


In [43]:
corrDf = csData.corr()

In [44]:
corrDf

,balance,stock,label_new
balance,1.000000,0.565942,0.883144
stock,0.565942,1.000000,0.824174
label_new,0.883144,0.824174,1.000000


In [45]:
corrStd = 0.5

In [51]:
features = list (corrDf.loc[ (abs(corrDf[labelColumn]) > corrStd) & (corrDf[labelColumn] != 1)].index )

In [52]:
features

['balance', 'stock']

##  2- 2 데이터 분리

#### 특성 선정 시에서는, 정답지와 관련이 있는 피쳐를 설정해야 하기에, 해당 컬럼값을 숫자로 바꿔서 계산을 해서 상관관계 여부를 찾은거고, 이후에 Label_new는 필요없다. 
#### 다시 복구해준다.

'label'

In [72]:
labelColumn = "label"

In [64]:
## loc[:] 는 전체라는 뜻
trainingDataFeatures, testDataFeatures, trainingDataLabel, testDataLabe = train_test_split( csData.loc[:, features],
                  csData.loc[:, labelColumn],
                  random_state=1,
                  test_size=0.2 )

In [65]:
trainingDataFeatures.shape

(16000, 2)

In [66]:
testDataFeatures.shape

(4000, 2)

In [67]:
trainingDataLabel.shape

(16000,)

In [68]:
testDataLabe.shape

(4000,)

## 3. 모델 선언 / 학습

In [69]:
## C는 마진. 하이퍼 파라미터 튜닝으로 자주 사용
model_svm = SVC(random_state = 5)


## 4. 예측

In [74]:
predictSvm = model_svm.predict(testDataFeatures)

## 5. 데이터 정리

In [77]:
## 원래 데이터에서 답안지의 데이터 뽑기
testDataAll = csData.loc [testDataLabe.index]

In [78]:
testDataAll["predict_svm"] = predictSvm

In [79]:
testDataAll

,balance,stock,label,label_new,predict_svm
11456,744000000,38000000,diamond,1,diamond
16528,724000000,32000000,diamond,1,diamond
3253,704000000,27333333,diamond,1,diamond
18614,240000000,30500000,normal,0,normal
1544,258000000,28500000,normal,0,normal
...,...,...,...,...,...
6375,640000000,48666666,diamond,1,diamond
14837,652000000,26000000,diamond,1,diamond
3931,564000000,45333333,diamond,1,diamond
18266,528000000,46000000,diamond,1,diamond


## 6. 정확도 검증

In [80]:
accuracy = accuracy_score(y_true = testDataAll.label,
              y_pred = testDataAll.predict_svm)

In [81]:
accuracy

0.995

In [84]:
testDataAll.to_csv("d:/svc_result.csv", index=False)

### 해당 모델로 미래 예측.. sav파일과 py파일이 둘다 필요합니다.

In [87]:
import pickle

filename = 'finalized_model_svc.sav'
pickle.dump(model_svm, open(filename, 'wb'))